### This file is used to retrieve weather data along with NEE from the ameriFLUX maize-soybean datasets

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from scipy.stats import pearsonr
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import os
import warnings
from sklearn import preprocessing
import scipy.stats
warnings.filterwarnings('ignore')

In [2]:
# loads all data sets into a dict
def load_datasets(dirs: list, load_path: str) -> dict:
    files = ['soil_c','surf_water','flux_soc','soil_water','n_flux','p_flux','temp', 
         'plant_c','plant_n','plant_p','canopcy_c','plant_stress','photosynthesis','plant_growth']
    
    #files.append('soil_temp', 'canopy_temp') missing 
    datasets = {}
    
    for dr in dirs:
        csv_list = []
        path = '../datasets/' + dr +  '/' + load_path
        for f in files:
            df = pd.read_csv(os.path.join(path,f + '.csv'))
            df.drop(df.columns[0], axis=1)

            csv_list.append(df)


        data_dict = {}
        for i in range (len(csv_list)):
            data_dict[files[i]] = csv_list[i]

        datasets[dr] = data_dict

    return datasets

dirs = ['warm_temp_maize_soybean_irrigated', 'warm_temp_maize-soybean_dryland', 'cool_temp_maize_soybean']

datasets = load_datasets(dirs, 'csv_outs/with_plant_soil_details/')

In [3]:
def rename_dupes(suffix: str, df: pd.DataFrame, dupes: list) -> pd.DataFrame:
    for col in df.columns:
        if col in dupes:
            df.rename(columns={col: col + suffix}, inplace=True)
    return df

def average_numbered_columns(df):
    numbered_cols = [col for col in df.columns if '_' in col and col.split('_')[-1].isdigit()]

    col_groups = {}
    for col in numbered_cols:
        prefix = '_'.join(col.split('_')[:-1])
        if prefix not in col_groups:
            col_groups[prefix] = []
        col_groups[prefix].append(col)

    # calculate averages and add new columns
    for prefix, cols in col_groups.items():
        avg_col_name = prefix
        avg_col_values = df[cols].mean(axis=1)
        df[avg_col_name] = avg_col_values

    # drop numbered columns
    df = df.drop(columns=numbered_cols)

    return df

# turn all csv's to one dataframe
def to_pd(df: dict, handle_dupes: bool, flatten_num_cols: bool) -> pd.DataFrame:
    x = pd.DataFrame()
    for file_name in df:
        cur = df[file_name]
            
        x = pd.concat([x, df[file_name]], axis = 1)
        
    cheeky_col = 'unnamed.1'
    cheeky_col2 = 'Unnamed: 0'
    if cheeky_col in x.columns:
        x = x.drop([cheeky_col], axis=1)
    elif cheeky_col2 in x.columns:
         x = x.drop([cheeky_col2], axis=1)
    x = x.drop(['DATE'], axis=1)

    if flatten_num_cols:
        x = average_numbered_columns(x)
        
    x = x.loc[:,~x.columns.duplicated()].copy()
    
    one_hot = pd.get_dummies(x['GROWTH_STG'])
    x= x.drop('GROWTH_STG',axis = 1)
    # Join the encoded df
    x = x.join(one_hot)

    x.columns = x.columns.str.translate("".maketrans({"[":"{", "]":"}","<":"^"}))
    
    return x

df_dry = to_pd(datasets['warm_temp_maize-soybean_dryland'], True, True)
df_irr = to_pd(datasets['warm_temp_maize_soybean_irrigated'], True, True)
df_cool = to_pd(datasets['cool_temp_maize_soybean'], True, True)

# NEE = GPP - ER:
#GPP = GROSS PRIMARY PRODUCTION (TOTAL C INTAKE) 
#ER = total C uptake =  ECO_RH + ECO_RA =  autotrophic + heterotrophic respiration 
#NPP = GPP + ECO_RA
df_dry['NEE'] = df_dry['ECO_NPP'] - df_dry['ECO_RH']
df_irr['NEE'] = df_irr['ECO_NPP'] - df_irr['ECO_RH']
df_cool['NEE'] = df_cool['ECO_NPP'] - df_cool['ECO_RH']
nee = df_irr['NEE']
#y = df[target].copy()
#df = df.drop(target, axis=1)

In [ ]:

avg_w_ins = ['DOY', 'RADN', 'TMAX_AIR', 'TMIN_AIR',  'WIND',
       'D_W_PRECN','TMAX_SOIL', 'TMIN_SOIL', 'HMAX_AIR', 'HMIN_AIR']
df_irr = df_irr[avg_w_ins][:2192].reset_index()

nee = nee[:2192]

In [4]:
df_irr['DOY'] = df_irr['DOY'].astype(int)
df_irr['DOY'] = df_irr['DOY'] + 1

In [5]:
df_irr['ECO_NPP_D'] = df_irr['ECO_NPP'] / df_irr['DOY']
df_irr['ECO_RH_D'] = df_irr['ECO_RH'] / df_irr['DOY']

df_irr['NEE_D'] = df_irr['ECO_NPP_D'] - df_irr['ECO_RH_D']

nee_d = df_irr['NEE_D']

In [7]:
mdays=[31,28,31,30,31,30,31,31,30,31,30,31]
def isleap(year):
    leap=False
    if year % 4 == 0 and year % 100 != 0:
        leap=True
    elif year % 100 == 0:
        leap=False
    elif year % 400 ==0:
        leap=True
    else:
        leap=False
    return leap
def dayofyear(year,month,day):
    doy=0
    for mm in range(0,month):
        days=mdays[mm]
        if mm==1:
            if isleap(year):
                days=29
        for dd in range(0,days):
            if mm < month-1:
                doy=doy+1
            else:
                if dd < day:
                    doy=doy+1
    return doy

### Daily ameriflux data for site US-Ne1 

In [8]:
df = pd.read_csv('../datasets/AMF_US-Ne1_FLUXNET_FULLSET_DD_2001-2020_3-5.csv')

mask = (df['TIMESTAMP'] >= 20020101) & (df['TIMESTAMP'] < 20150101)

df = df.loc[mask].reset_index()


filter_col = [col for col in df if col.startswith('NEE') and not col.endswith('QC')]    
nee_df = df[filter_col]

df['TIMESTAMP'] = pd.to_datetime(df['TIMESTAMP'], format='%Y%m%d')
df['DOY'] = 0
for i in range( df.shape[0]):
    df['DOY'][i] = dayofyear(df['TIMESTAMP'][i].year, df['TIMESTAMP'][i].month, df['TIMESTAMP'][i].day)

#df.drop('Index', axis=1)

### Introduce two measurements Jensen Shannon distance, and  pearson's correlation coefficient distance to find the most similar observed NEE 
#### Linear and non-linear

In [46]:
def jensen_shannon_distance(p, q):
    """
    method to compute the Jenson-Shannon Distance 
    between two probability distributions
    """

    # convert the vectors into numpy arrays in case that they aren't
    p = np.array(p)
    q = np.array(q)

    # calculate m
    m = (p + q) / 2

    # compute Jensen Shannon Divergence
    divergence = (scipy.stats.entropy(p, m) + scipy.stats.entropy(q, m)) / 2

    # compute the Jensen Shannon Distance
    distance = np.sqrt(divergence)

    return distance


def pearson(p,q):
    return np.corrcoef(p, q)
    

### jensen shannon distance, Pearson corr, cosine similarity and K means
#### Used to find the most similar probability distributions between the variations of weather data from observed, and ecosys

In [47]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import silhouette_score

def get_most_similar(y, df, normalized: False) -> dict:
    scores = {}
    smallest_residual  = 99999
    biggest_corr = -9999
    col_corr = ''
    col_res = ''
    
    y_norm = pd.Series(preprocessing.normalize([y])[0], name='NEE')

    for col in df:
        if normalized:
            y = y_norm
            df[col] = preprocessing.normalize([df[col]])[0]
            
        sim = jensen_shannon_distance(y, df[col])
        corr_coef = pearson(y, df[col]).mean()
        
        if sim  < smallest_residual:
            smallest_residual = sim
            col_res = col
            
        if corr_coef > biggest_corr:
            col_corr = col
            biggest_corr = corr_coef
            
    scores["JSD"] = {col_res: smallest_residual}
    scores["PRSN"] = {col_corr: biggest_corr}
    scores["Cosine similarity"] = cosine_sim(df, y)
    scores["K-Means"] = find_kmeans(df, y)
    
    return scores


def cosine_sim(X, y):
    similarities = {}
    y_reshaped = y.values.reshape(1, -1)  # Reshape y
    for col in X.columns:
        col_reshaped = X[col].values.reshape(1, -1)  # Reshape column
        similarity = cosine_similarity(col_reshaped, y_reshaped)
        similarities[col] = similarity[0][0]
    most_similar_col = max(similarities, key=similarities.get)
    most_similar_col_score = similarities[most_similar_col]
    return {most_similar_col: most_similar_col_score}

def find_kmeans(X, y):
    kmeans = KMeans(n_clusters=6, random_state=0).fit(X.T)
    y_cluster = kmeans.predict(y.values.reshape(1, -1))
    most_similar_col_idx = np.where(kmeans.labels_ == y_cluster)[0][0]
    return X.columns[most_similar_col_idx]


In [10]:
ext = pd.read_csv('../datasets/extractedUSNEE_CUT_05_NIGHT.csv')

In [11]:
ext.describe()

,Unnamed: 0,TMIN_AIR,TMAX_SOIL,TMAX_AIR,RADN,WIND,D_W_PRECN,TMIN_SOIL,NEE
count,2192.000000,2192.000000,2192.000000,2192.000000,2192.000000,2192.000000,2192.000000,2192.000000,2192.000000
mean,6208.500000,2.536832,12.038186,12.508763,178.459320,2.985406,2.618802,-9268.723397,2.945288
std,632.920216,1.480891,8.446921,11.586313,94.545238,1.100184,6.626027,2603.085535,3.265968
min,5113.000000,0.052000,-0.935000,-21.228000,6.200000,0.779000,0.000000,-9999.000000,-9.409830
25%,5660.750000,1.405750,3.415500,3.598000,103.761500,2.182250,0.000000,-9999.000000,0.601852
50%,6208.500000,2.220000,11.945500,13.953000,169.787500,2.822000,0.046000,-9999.000000,1.362580
75%,6756.250000,3.399500,20.584500,22.664500,258.478500,3.625250,1.425000,-9999.000000,5.305025
max,7304.000000,9.359000,26.569000,32.267000,374.446000,7.464000,84.902000,16.813000,14.375300


In [48]:
filter_col = [col for col in df if col.startswith('NEE') and not col.endswith('QC')]    
nee_df = df[filter_col]

### Avg wind M s^-1 is 300 in simulated data, so reducing it by 100 factor 

In [49]:
df_irr['WIND'] = df_irr['WIND']/ 100

In [50]:
df_irr['WIND']

0       3.31524
1       2.47068
2       2.45952
3       3.76056
4       3.67992
         ...   
2187    1.48104
2188    2.03364
2189    4.27104
2190    2.22876
2191    2.22876
Name: WIND, Length: 2192, dtype: float64

In [51]:
similarities_to_find = ['RADN', 'TMAX_AIR', 'TMIN_AIR',  'WIND',
       'D_W_PRECN','TMAX_SOIL', 'TMIN_SOIL']

for col in similarities_to_find:
    print("Similarity for " + col + ":") 
    print(get_most_similar(df_irr[col], df, True))

Similarity for RADN:
{'JSD': {'SW_IN_POT': 0.1271263645923741}, 'PRSN': {'SW_IN_POT': 0.8901600167911246}, 'Cosine similarity': {'SW_IN_POT': 0.9580015543529974}, 'K-Means': 'index'}
Similarity for TMAX_AIR:
{'JSD': {'': 99999}, 'PRSN': {'TS_F_MDS_4': 0.9162869304155195}, 'Cosine similarity': {'TS_F_MDS_7': 0.9343146027310274}, 'K-Means': 'TA_F_MDS'}
Similarity for TMIN_AIR:
{'JSD': {'': 99999}, 'PRSN': {'TS_F_MDS_4': 0.934054023616878}, 'Cosine similarity': {'TA_ERA_NIGHT': 0.8212129266913064}, 'K-Means': 'TA_F_MDS'}
Similarity for WIND:
{'JSD': {'SWC_F_MDS_3': 0.13224455944025232}, 'PRSN': {'NEE_CUT_05_DAY': 0.6557638726457695}, 'Cosine similarity': {'SWC_F_MDS_3': 0.9328949966289855}, 'K-Means': 'index'}
Similarity for D_W_PRECN:
{'JSD': {'GPP_DT_CUT_SE': 0.6683177752443282}, 'PRSN': {'RECO_NT_VUT_95': 0.615246704251489}, 'Cosine similarity': {'RECO_NT_VUT_95': 0.37230943847804804}, 'K-Means': 'TA_F_MDS'}
Similarity for TMAX_SOIL:
{'JSD': {'SW_IN_POT': 0.14383686289778708}, 'PRSN': 

In [53]:
nee.describe()

count    2192.000000
mean        3.031039
std         2.356084
min         0.120654
25%         0.533827
50%         2.866796
75%         5.170090
max         7.178794
Name: NEE_D, dtype: float64

In [54]:
nee_df

,NEE_CUT_REF,NEE_VUT_REF,NEE_CUT_REF_RANDUNC,NEE_VUT_REF_RANDUNC,NEE_CUT_REF_JOINTUNC,NEE_VUT_REF_JOINTUNC,NEE_CUT_USTAR50,NEE_VUT_USTAR50,NEE_CUT_USTAR50_RANDUNC,NEE_VUT_USTAR50_RANDUNC,...,NEE_CUT_75_DAY,NEE_CUT_84_DAY,NEE_CUT_95_DAY,NEE_VUT_05_DAY,NEE_VUT_16_DAY,NEE_VUT_25_DAY,NEE_VUT_50_DAY,NEE_VUT_75_DAY,NEE_VUT_84_DAY,NEE_VUT_95_DAY
0,0.868608,0.827235,0.079407,0.099276,0.104149,0.119726,0.905505,0.827235,0.078600,0.099276,...,0.969487,0.977848,1.042640,0.886008,0.886008,0.886008,0.896061,0.964485,0.971482,1.016640
1,0.753910,0.651776,0.080727,0.087668,0.160519,0.207747,0.753910,0.651776,0.080223,0.087668,...,0.921082,0.952865,0.961961,0.468075,0.474439,0.482874,0.482874,0.677872,0.884742,0.973945
2,0.737119,0.613439,0.103860,0.105802,0.185087,0.162311,0.746343,0.613439,0.103537,0.105802,...,0.924795,0.927869,1.189370,0.386000,0.442121,0.516364,0.560909,0.762462,0.925366,0.933977
3,0.619648,0.614550,0.072333,0.080011,0.075251,0.082592,0.619648,0.614550,0.071903,0.080011,...,0.494545,0.496511,0.497348,0.487595,0.489328,0.491212,0.491341,0.492129,0.494545,0.497424
4,0.771991,0.717703,0.101710,0.111163,0.114324,0.116168,0.771991,0.717703,0.101987,0.111163,...,0.662566,0.663264,0.723636,0.595455,0.595455,0.613013,0.614291,0.614954,0.621095,0.665977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2187,0.996049,0.994410,0.164621,0.156168,0.167872,0.161601,0.996049,0.994410,0.155551,0.156168,...,1.009000,1.095220,1.139060,1.009000,1.009000,1.009000,1.009000,1.009000,1.095220,1.140390
2188,0.749068,0.709396,0.167811,0.161756,0.176605,0.171921,0.749068,0.709396,0.160450,0.161756,...,0.847306,0.896056,1.025360,0.736000,0.736000,0.736000,0.736000,0.827750,0.892889,1.022910
2189,0.708641,0.693536,0.161957,0.159840,0.186515,0.196185,0.708641,0.693536,0.159769,0.159840,...,0.786000,0.805250,0.820361,0.729000,0.729000,0.729000,0.751067,0.786000,0.803317,0.820361
2190,0.538013,0.503327,0.201586,0.195956,0.246981,0.224516,0.538013,0.503327,0.200398,0.195956,...,0.937865,0.952562,0.970613,0.613874,0.613874,0.649018,0.678714,0.935683,0.939242,0.972448


In [60]:
nee.describe()

count    2922.000000
mean      783.362736
std       728.606997
min         0.127113
25%        52.783893
50%       527.658315
75%      1512.628125
max      1911.120400
Name: NEE, dtype: float64

In [61]:
nee_d.describe()

count    2922.000000
mean        3.077716
std         2.353805
min         0.129128
25%         0.574323
50%         2.949824
75%         5.209081
max         7.192862
Name: NEE_D, dtype: float64

In [63]:
get_most_similar(nee[:2192], nee_df, True)

{'JSD': {'NEE_VUT_REF_NIGHT_SD': 0.4564486398641424},
 'PRSN': {'NEE_CUT_05_NIGHT': 0.5844031376157531},
 'Cosine similarity': {'NEE_CUT_75_NIGHT': 0.5883954132109497},
 'K-Means': 'NEE_CUT_REF_NIGHT'}

In [67]:
get_most_similar(nee_d[:2192], nee_df, True)

{'JSD': {'NEE_VUT_REF_NIGHT_SD': 0.3951475827701259},
 'PRSN': {'NEE_VUT_16_NIGHT': 0.6829585823441287},
 'Cosine similarity': {'NEE_CUT_75_NIGHT': 0.7102379814996385},
 'K-Means': 'NEE_CUT_REF_NIGHT'}

In [10]:
df_irr = pd.read_csv('../datasets/warm_temp_maize_soybean_irrigated/full_csv_outs/with_plant_soil_details/temp.csv')

In [14]:
def average_numbered_columns(df):
    numbered_cols = [col for col in df.columns if '_' in col and col.split('_')[-1].isdigit()]

    col_groups = {}
    for col in numbered_cols:
        prefix = '_'.join(col.split('_')[:-1])
        if prefix not in col_groups:
            col_groups[prefix] = []
        col_groups[prefix].append(col)

    # calculate averages and add new columns
    for prefix, cols in col_groups.items():
        avg_col_name = prefix
        avg_col_values = df[cols].mean(axis=1)
        df[avg_col_name] = avg_col_values

    # drop numbered columns
    df = df.drop(columns=numbered_cols)

    return df

In [15]:
df_irr = average_numbered_columns(df_irr)

In [16]:
df_irr.columns

Index(['Unnamed: 0', 'DOY', 'DATE', 'RADN', 'TMAX_AIR', 'TMIN_AIR', 'HMAX_AIR',
       'HMIN_AIR', 'WIND', 'D_W_PRECN', 'TMAX_LITTER', 'TMIN_LITTER',
       'TTL_SALT_DISCHG', 'TMAX_SOIL', 'TMIN_SOIL', 'ECND'],
      dtype='object')

In [17]:
extracted= {
'DOY' : df['DOY'],
'HMAX_AIR' :  df_irr['HMAX_AIR'], # dataset missing air humidity:/ 
'TMIN_AIR' : df['TA_ERA_NIGHT'], #TA_F_MDS also works
'TMAX_SOIL' : df['TS_F_MDS_7'] , #cosine and pearson agree
'HMIN_AIR' :  df_irr['HMIN_AIR'], #missing air humidity 
'TMAX_AIR' : df['TA_F_MDS'] ,
'RADN' : df['SW_IN_POT'],
'WIND' : df['WS_F'] ,
'D_W_PRECN' : df['P_F'], 
'TMIN_SOIL' : df_irr['TMIN_SOIL'], #observed data missing min soil temp
'NEE' : df['NEE_CUT_REF_NIGHT']
}

extracted_df = pd.DataFrame(extracted)

extracted_df.to_csv('../datasets/TRAINextractedNEE_CUT_REF_NIGHTobsfeats.csv')


In [77]:
extracted_df

,DOY,HMAX_AIR,TMIN_AIR,TMAX_SOIL,HMIN_AIR,TMAX_AIR,RADN,WIND,D_W_PRECN,TMIN_SOIL,NEE
0,0.520833,0.630569,-0.012605,0.002943,0.435766,-0.008448,0.009351,0.017072,0.000000,3.167734,0.003631
1,1.520833,0.630147,-0.009287,0.003120,0.349557,-0.007237,0.009390,0.010835,0.000000,2.970467,0.003638
2,2.520833,0.796492,-0.011451,0.002920,0.324438,-0.009198,0.009432,0.021395,0.004675,2.820677,0.003394
3,3.520833,0.897199,-0.022331,0.001994,0.501948,-0.021263,0.009477,0.027115,0.003280,3.144463,0.003015
4,4.520833,0.702206,-0.016221,0.002278,0.384053,-0.014926,0.009527,0.020603,0.001424,3.087650,0.003700
...,...,...,...,...,...,...,...,...,...,...,...
2187,361.520833,0.835153,-0.003106,0.002748,0.441234,-0.000863,0.009234,0.023302,0.002816,3.194780,0.004216
2188,362.520833,0.835904,-0.006845,0.002588,0.423138,-0.005282,0.009258,0.012708,0.000003,3.187274,0.003101
2189,363.520833,0.784312,-0.005813,0.002437,0.363440,-0.005136,0.009285,0.020563,0.046197,3.110832,0.003078
2190,364.520833,0.696367,-0.012196,0.002307,0.271404,-0.008375,0.009316,0.018173,0.000000,2.867376,0.002089


In [99]:
biggest_mean_for('TA', df)

'TA_F_MDS_DAY'

In [100]:
smallest_mean_for('TA', df)

'TA_F_DAY_QC'

In [14]:
filter_col = [col for col in df if col.startswith('TS') and not col.endswith('QC')]    

In [15]:
filter_col

['TS_F_MDS_1',
 'TS_F_MDS_2',
 'TS_F_MDS_3',
 'TS_F_MDS_4',
 'TS_F_MDS_5',
 'TS_F_MDS_6',
 'TS_F_MDS_7',
 'TS_F_MDS_8']

In [12]:
smallest_mean_for('TS', df)

'TS_F_MDS_3_QC'

In [11]:
df[filter_col].mean()

TS_F_MDS_1       11.348499
TS_F_MDS_2       11.334961
TS_F_MDS_3       12.403337
TS_F_MDS_4       11.385723
TS_F_MDS_5       11.384166
TS_F_MDS_6             NaN
TS_F_MDS_7       11.445622
TS_F_MDS_8       11.366537
TS_F_MDS_1_QC     0.999620
TS_F_MDS_2_QC     0.999563
TS_F_MDS_3_QC     0.876141
TS_F_MDS_4_QC     0.999639
TS_F_MDS_5_QC     0.999391
TS_F_MDS_6_QC          NaN
TS_F_MDS_7_QC     0.984824
TS_F_MDS_8_QC     0.999468
dtype: float64